In [4]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from functions import *

In [57]:
max_price = 900

In [61]:
towns = ['sabadell']
property = "alquiler"
ascensor = "-ascensor"
maximum_price = "pmax="+str(max_price)

In [107]:
# DataFrame that we will fill in with web data
dct = {'name': [],
       'town':[],
       'area': [],
       'neighborhood': [],
       'geo_town': [],
       'm2': [],
       'n_rooms': [],
       'n_bath': [],
       'price_m2': ['a','b'],
       'description': [],
       'price': [],
       'opportunity': [],
       'price_reduction': [],
       'last_update': [],
       'url': [],
       'lift': []}

# Data Scraping begins
for t in towns:
    url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        # Getting the number of properties for that town to see how many pages do we need to scrape.
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []
        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,3))
            url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Getting the information from each property.
        for i,properties in enumerate(properties):  
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""
            lift_temp = ""
            
            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    lift_temp = 'yes'

                    # Appending temporary variables features to their corresponding list.
                    dct['name'].append(name_temp)
                    dct['town'].append(town_temp)
                    dct['area'].append(area_temp)
                    dct['neighborhood'].append(neighborhood_temp)
                    dct['geo_town'].append(geo_town_temp)
                    dct['m2'].append(m2_temp)
                    dct['n_rooms'].append(n_rooms_temp)
                    dct['n_bath'].append(n_bath_temp)
                    dct['price_m2'].append(price_m2_temp)
                    dct['price'].append(price_temp)
                    dct['opportunity'].append(opportunity_temp)
                    dct['price_reduction'].append(price_reduction_temp)
                    dct['description'].append(description_temp)
                    dct['last_update'].append(last_update_temp)
                    dct['url'].append(url_temp)
                    dct['lift'].append(lift_temp)
                    
                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    print('error')
        
        x = min(map(len, dct.values()))
        df = pd.DataFrame({k:v[:x] for k,v in dct.items()})
        
        #Cleaning the final df
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]
    except:
        print("no properties found at: ", url)

In [108]:
df

,name,town,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,description,price,opportunity,price_reduction,last_update,url,lift
0,Alquiler Piso en Centre. Pis en lloguer al c...,Sabadell,Sabadell,Centre,sabadell,90,2,2,a,"GESTIONS IMMOBILIÀRIES NAYACH comercialitza, P...",877,no,0,[7],https://www.habitaclia.com/alquiler-piso-pis_e...,yes
1,Alquiler Piso en Creu Alta. Piso semi nuevo ...,Sabadell,Sabadell,Creu Alta,sabadell,47,1,1,b,"Piso en la creu alta de 47 m2 según catastro, ...",725,no,0,[6],https://www.habitaclia.com/alquiler-piso-semi_...,yes
2,Alquiler Piso Carrer borrell,Sabadell,Sabadell,Eixample-Sant Oleguer,sabadell,54,2,1,9,SIN COMISIÓN DE AGENCIA Vivienda seminueva de...,726,no,0,[],https://www.habitaclia.com/alquiler-piso-eixam...,yes
3,Alquiler Piso Can rull. Para entrar a vivir!!,Sabadell,Sabadell,Can Rull,sabadell,80,2,1,15,NOSTRALLAR alquila esta vivienda de 2 dormitor...,750,no,0,[9],https://www.habitaclia.com/alquiler-piso-para_...,yes
4,Alquiler Piso en Gràcia. Piso de obra nueva ...,Sabadell,Sabadell,Gràcia,sabadell,60,2,1,13,Piso con salón comedor luminoso y salida a bal...,790,yes,50,[3],https://www.habitaclia.com/alquiler-piso-de_ob...,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Alquiler Piso Calle espirall. Dúplex en alqui...,Sabadell,Sabadell,Centre,sabadell,165,3,2,7,PROMOCIÓN DE NOVIEMBRE: 1 MES GRATIS si reserv...,930,no,0,[4],https://www.habitaclia.com/fa29468701,yes
80,Alquiler Piso Collsalarca. Piso en alquiler e...,Sabadell,Sabadell,Ca n´Oriac,sabadell,56,3,1,13,PROMOCIÓN DE NOVIEMBRE: 1 MES GRATIS si reserv...,692,no,0,[3],https://www.habitaclia.com/fa29302289,yes
81,Alquiler Piso Calle saturn. Piso en alquiler ...,Sabadell,Sabadell,Gràcia,sabadell,68,4,1,5,PROMOCIÓN DE NOVIEMBRE: 1 MES GRATIS si reserv...,800,no,0,[3],https://www.habitaclia.com/fa29066694,yes
82,Alquiler Piso Calle eslava. Piso en alquiler ...,Sabadell,Sabadell,Can Rull,sabadell,53,2,1,12,PROMOCIÓN DE NOVIEMBRE: 1 MES GRATIS si reserv...,710,no,0,[3],https://www.habitaclia.com/fa29066692,yes
